In [1]:
import re
import pandas as pd
import numpy as np
import gspread
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
gc = gspread.service_account(filename='/home/gbotemi/Documents/gbotemi_bolarinwa/gspread-project/secret.json') # use your google service file account here
file = "https://docs.google.com/spreadsheets/d/1SUlcukpgxf6pfFQbj6DKQZXSlOhyBOSuBdbmQ2ZbhTM/edit?usp=sharing"

sh = gc.open_by_url(file)
worksheet = sh.get_worksheet(1)

In [3]:
data = pd.DataFrame(worksheet.get_all_records())

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   S#                       496 non-null    int64 
 1   Date                     496 non-null    object
 2   Islamic Date             496 non-null    object
 3   Blast Day Type           496 non-null    object
 4   Holiday Type             496 non-null    object
 5   Time                     496 non-null    object
 6   City                     496 non-null    object
 7   Latitude                 496 non-null    object
 8   Longitude                496 non-null    object
 9   Province                 496 non-null    object
 10  Location                 496 non-null    object
 11  Location Category        496 non-null    object
 12  Location Sensitivity     496 non-null    object
 13  Open/Closed Space        496 non-null    object
 14  Influencing Event/Event  496 non-null    o

In [5]:
data.head()

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,Location,Location Category,Location Sensitivity,Open/Closed Space,Influencing Event/Event,Target Type,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,N/A,Islamabad,33.718,73.0718,Capital,Egyptian Embassy,Foreign,High,Closed,,Foreigner,None,14,15,,60,2,,,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,,N/A,Karachi,24.9918,66.9911,Sindh,office of Nawa-e-Waqt,Office Building,Low,Closed,,Media,None,,3,,3,1,,,23.77,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,,7:45 AM,Karachi,24.9918,66.9911,Sindh,Pakistan Navy bus Parked outside Five Star She...,Hotel,Medium,Closed,,Foreigner,Christian,13,15,20,40,1,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.46,88.628
3,4,Friday-June 14-2002,3 Raby` al-THaany 1423 A.H,Working Day,,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,US Consulate Civil Lines Area,Foreign,High,Closed,,Foreigner,Christian,,12,,51,1,NA,NA,31.43,88.574
4,5,Friday-July 4-2003,4 Jumaada al-awal 1424 A.H,Working Day,,N/A,Quetta,30.2095,67.0182,Baluchistan,Imambargah MeCongy Road Quetta,Religious,Medium,Closed,during Friday prayer,Religious,Shiite,44,47,,65,1,NA,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.12,91.616


In [6]:
data = data.replace("", np.nan)
data = data.replace("N/A", np.nan)
data = data.replace("NA", np.nan)
data.columns = data.columns.str.lower() # lower columns values

In [7]:
data['date'] = pd.to_datetime(data['date'], errors='coerce')	
data['time'] = pd.to_datetime(data['time'], errors='coerce').dt.time

In [8]:
data["explosive weight (max)"] = data["explosive weight (max)"].replace(r"to", "-", regex=True)
data["explosive weight (max)"] = data["explosive weight (max)"].str.replace(r"[^0-9-]+", "", regex=True)
data["explosive weight (max)"] = data["explosive weight (max)"].replace(r"\d*-", "", regex=True).replace("", np.nan).astype("float")

In [9]:
data["injured max"] = data["injured max"].replace(r"[^A-z ]", "", regex=True)
data["injured max"] = data["injured max"].replace(r"[A-z]", "", regex=True)
data["injured max"] = data["injured max"].replace("", np.nan).astype("float")

In [10]:
data.columns

Index(['s#', 'date', 'islamic date', 'blast day type', 'holiday type', 'time',
       'city', 'latitude', 'longitude', 'province', 'location',
       'location category', 'location sensitivity', 'open/closed space',
       'influencing event/event', 'target type', 'targeted sect if any',
       'killed min', 'killed max', 'injured min', 'injured max',
       'no. of suicide blasts', 'explosive weight (max)', 'hospital names',
       'temperature(c)', 'temperature(f)'],
      dtype='object')

In [11]:
data.to_csv("data/modified_inconsistent_data.csv", index=False) #export cleaned data